# Alphametics Solver

[Alphametics](https://en.wikipedia.org/wiki/Verbal_arithmetic) is a type of cryptarithm in which a set of words is written down in the form of a long addition sum or some other mathematical problem. The objective is to replace the letters of the alphabet with decimal digits to make a valid arithmetic sum.

For this kata, your objective is to write a function that accepts an alphametic equation in the form of a single-line string and returns a valid arithmetic equation in the form of a single-line string.
Test Examples

```
INPUT:    "SEND + MORE = MONEY"
SOLUTION: "9567 + 1085 = 10652"

INPUT:    "ELEVEN + NINE + FIVE + FIVE = THIRTY"
SOLUTION: "797275 + 5057 + 4027 + 4027 = 810386"
```

Some puzzles may have multiple valid solutions; your function only needs to return one

```
BIG + CAT = LION
403 + 679 = 1082
326 + 954 = 1280
304 + 758 = 1062
...etc.
```

Technical Details

* All alphabetic letters in the input will be uppercase
* Each unique letter may only be assigned to one unique digit
* As a corollary to the above, there will be a maximum of 10 unique letters in any given test
* No leading zeroes
* The equations will only deal with addition with multiple summands on the left side and one term on the right side
* The number of summands will range between 2 and 7, inclusive
* The length of each summand will range from 2 to 8 characters, inclusive
* All test cases will be valid and will have one or more possible solutions
* Full Test Suite: 15 fixed tests, 21 random tests for Python and Ruby / 18 random tests for JavaScript / 28 random tests for Go and C# / 136 random tests for Java / 72 random tests for Kotlin
* Optimize your code -- a naive, brute-force algorithm may time out before the first test completes
* For JavaScript, module and require are disabled, and most prototypes are frozen (except Array and Function)
* For Python, module imports are prohibited
* Python users: Due to the performance of the Python runner, it is advised to attempt solving this kata in another language besides Python.
* Use Python 3.6+ for the Python translation

Source: [Alphametics Solver](https://www.codewars.com/kata/5b5fe164b88263ad3d00250b)

## Brute force

Lets not pay attention to the description, brute force it is!

The algorithm, is calculate the multiplayer for each letter (based on the positions in the word or in equation).

In [1]:
import itertools

BASE = 10

def word_to_multiplayers(word):
    multiplayer = 1
    for letter in reversed(word):
        yield (letter, multiplayer)
        multiplayer *= BASE

def digit_permutations(how_many):
    yield from itertools.permutations(range(0, BASE), how_many)

def add_letters(checking_table, word, a):
    for letter, multiplayer in word_to_multiplayers(word):
        tmp = checking_table.get(letter, [])
        tmp.append(a * multiplayer)
        checking_table[letter] = tmp

In [2]:
%%time
    
def alphametics(puzzle):
    leftside, rightside = puzzle.split(' = ')

    checking_table = {}
    for words in leftside.split(' + '):
        add_letters(checking_table, words, 1)
    
    add_letters(checking_table, rightside, -1)
        
    letter_number = len(checking_table.keys())
    for digits in digit_permutations(letter_number):
        value_sum = sum(
            sum(digit * multiplayer for multiplayer in multiplayers)
            for digit, multiplayers in zip(digits, checking_table.values()))

        if value_sum != 0:
            continue

        result = puzzle
        for digit, letter in zip(digits, checking_table.keys()):
            result = result.replace(letter, str(digit))

        return result

    raise 'No found'

print('Solution:', alphametics('SEND + MORE = MONEY'))

Solution: 3821 + 0468 = 04289
CPU times: user 1.12 s, sys: 3.83 ms, total: 1.12 s
Wall time: 1.12 s


This approach is working. Slowly, but working. Too slow and not working correctly, because it allow to leading zeros. It need to changed.

## Brute force with better bouncer

Lets check the last column of letter first. The idea is to reject incorrect permutation early.
By the last column I meant (in the example below) the letters *D*, *E* and *Y*.

```
  SEND
+ MORE
------
 MONEY
```

In [3]:
%%time
    
def alphametics(puzzle):
    leftside, rightside = puzzle.split(' = ')

    last_letters = set()
    checking_table = {}
    for word in leftside.split(' + '):
        add_letters(checking_table, word, 1)
        last_letters.add(word[-1])
    
    add_letters(checking_table, rightside, -1)
        
    letter_number = len(checking_table.keys())
    for digits in digit_permutations(letter_number):
        y = {k:d for d, k in zip(digits, checking_table.keys())}

        if sum(y[letter] for letter in last_letters) != (y[rightside[-1]] % 10):
            continue
        
        value_sum = sum(
            sum(digit * multiplayer for multiplayer in multiplayers)
            for digit, multiplayers in zip(digits, checking_table.values()))

        if value_sum != 0:
            continue

        result = puzzle
        for digit, letter in zip(digits, checking_table.keys()):
            result = result.replace(letter, str(digit))

        return result

    raise 'No found'

print('Solution:', alphametics('SEND + MORE = MONEY'))

Solution: 3821 + 0468 = 04289
CPU times: user 444 ms, sys: 0 ns, total: 444 ms
Wall time: 442 ms


The solution is almost three times faster, but still it is too slow for the CodeWars server. It also providing leading zeros, but it is not important - test cannot be done (timeout). Lets change the approach.

## Finding digits based on column

During work on that approach, I had to create the helper function for reverting solutions (see below, the `solution_to_digits` functions).


In [4]:
def solution_to_digits(puzzle, solution, letter_to_index):
    solution = {letter: int(digit) for letter, digit in zip(puzzle, solution) if letter not in ' +='}
    result = [0] * len(solution)
    for letter, digit in solution.items():
        result[letter_to_index[letter]] = digit

    return tuple(result)

The approach required new functions:

In [5]:
from collections import defaultdict

def check_columns(checking_table, digits):
    multiplayer = 1
    from_previous = 0

    multiplayers = sorted(multiplayer for multiplayer in checking_table.keys() if multiplayer > 0)
    for multiplayer in multiplayers:
        result = sum(digits[index] for index in checking_table[multiplayer]) + from_previous
        from_previous = result // BASE
        result %= BASE

        if result != digits[checking_table[-1 * multiplayer][0]]:
            return False
    
    multiplayer *= BASE
    if (-1 * multiplayer) not in checking_table:
        return from_previous == 0

    return from_previous == digits[checking_table[-1 * multiplayer][0]]

def build_checking_sets(puzzle):
    leftside, rightside = puzzle.split(' = ')
    all_letters = set(letter for letter in puzzle if letter not in ' +=')
    letter_to_index = {letter:i for i, letter in enumerate(all_letters)}
    not_zeros = set()

    checking_table = defaultdict(list)
    for word in leftside.split(' + '):
        for letter, multiplater in word_to_multiplayers(word):
            checking_table[multiplater].append(letter_to_index[letter])
        not_zeros.add(letter_to_index[word[0]])

    not_zeros.add(letter_to_index[rightside[0]])
    for letter, multiplater in word_to_multiplayers(rightside):
        checking_table[-1 * multiplater].append(letter_to_index[letter])
    
    return letter_to_index, checking_table, not_zeros

In [6]:
%%time

def alphametics(puzzle):
    letter_to_index, checking_table, not_zeros = build_checking_sets(puzzle)

    for digits in digit_permutations(len(letter_to_index)):
        if any(digits[index] == 0 for index in not_zeros):
            continue

        if not check_columns(checking_table, digits):
            continue

        for letter, index in letter_to_index.items():
            puzzle = puzzle.replace(letter, str(digits[index]))

        return puzzle

    raise 'No found'

print('Solution:', alphametics('SEND + MORE = MONEY'))

Solution: 9567 + 1085 = 10652
CPU times: user 2.63 s, sys: 3.69 ms, total: 2.63 s
Wall time: 2.66 s


The solution, is working correctly, but still is too slow. Is even slower than my original approach (but they cannot be compare, the first one, allow for leading zeros, this one not).